# LangChain Integration Examples

In [1]:
import os
from pprint import pprint as pp

os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

# The below environment variables are optional.

# The yaml config resolution priority is: args > env > cwd > package default.
# If you don't want to use either the package default (langgate/core/data/default_config.yaml)
# or a config in your cwd, set:
# os.environ["LANGGATE_CONFIG"] = "some_other_path_not_in_your_cwd/langgate_config.yaml"

# The models data resolution priority is: args > env > cwd > package default
# If you don't want to use either the package default (langgate/registry/data/default_models.json)
# or a models data file in your cwd, set:
# os.environ["LANGGATE_MODELS"] = "some_other_path_not_in_your_cwd/langgate_models.json"

# The .env file resolution priority is: args > env > cwd > None
# If you don't want to use either the package default or a .env file in your cwd, set:
# os.environ["LANGGATE_ENV_FILE"] = "some_other_path_not_in_your_cwd/.env"

## Basic SDK Usage

### Get Model Metadata

In [10]:
from langgate.sdk import LangGateLocal

client = LangGateLocal()


# LangGate allows us to register "virtual models" - models with specific parameters.
# `langgate_config.yaml` defines this `claude-sonnet-4-reasoning` model
# which is a wrapper around the `claude-sonnet-4-0` model,
# with specific parameters and metadata.
# In `langgate_config.yaml`, Anthropic is set as the inference service provider,
# but you could configure any backend API that offers the model, e.g. AWS Bedrock.
model_id = "anthropic/claude-sonnet-4-reasoning"

# get metadata for a model
model_info = await client.get_llm_info(model_id)

# returns a Pydantic model instance (langgate.core.models.LLMInfo)
pp(model_info.model_dump(exclude_none=True))

{'capabilities': {'supports_assistant_prefill': True,
                  'supports_prompt_caching': True,
                  'supports_reasoning': True,
                  'supports_response_schema': True,
                  'supports_tool_choice': True,
                  'supports_tools': True,
                  'supports_vision': True},
 'context_window': {'max_input_tokens': 200000, 'max_output_tokens': 64000},
 'costs': {'cache_creation_input_token_cost': '0.00000375',
           'cache_read_input_token_cost': Decimal('3E-7'),
           'input_cost_per_image': '0.0048',
           'input_cost_per_token': Decimal('0.000003'),
           'output_cost_per_token': Decimal('0.000015')},
 'description': 'Claude-4 Sonnet with reasoning capabilities.',
 'id': 'anthropic/claude-sonnet-4-reasoning',
 'name': 'Claude-4 Sonnet R',
 'provider': {'id': 'anthropic', 'name': 'Anthropic'},
 'provider_id': 'anthropic',
 'updated_dt': datetime.datetime(2025, 8, 18, 12, 58, 28, 591935, tzinfo=datetime.ti

### Transform Parameters

In [3]:
model_params = await client.get_params(model_id, {"temperature": 0.7, "stream": True})
pp(model_params)

('anthropic',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.anthropic.com',
  'model': 'claude-sonnet-4-0',
  'stream': True,
  'thinking': {'budget_tokens': 1024, 'type': 'enabled'}})


## LangChain Integration

In [ ]:
!uv pip install --resolution highest -U langchain langchain-openai langchain-anthropic

### Model Factory Pattern

In [5]:
from typing import Any
from pprint import pprint as pp

from langchain.chat_models.base import BaseChatModel
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langgate.sdk import LangGateLocal, LangGateLocalProtocol
from langgate.core.models import (
    # `ModelProviderId` is a string alias for better type safety
    ModelProviderId,
    # ids for common providers are included for convenience
    MODEL_PROVIDER_OPENAI,
    MODEL_PROVIDER_ANTHROPIC,
)

# Map providers to model classes
MODEL_CLASS_MAP: dict[ModelProviderId, type[BaseChatModel]] = {
    MODEL_PROVIDER_OPENAI: ChatOpenAI,
    MODEL_PROVIDER_ANTHROPIC: ChatAnthropic,
}


class ModelFactory:
    """
    Factory for creating a Langchain `BaseChatModel` instance
    with paramaters from LangGate.
    """

    def __init__(self, langgate_client: LangGateLocalProtocol | None = None):
        self.langgate_client = langgate_client or LangGateLocal()

    async def create_model(
        self, model_id: str, input_params: dict[str, Any] | None = None
    ) -> tuple[BaseChatModel, dict[str, Any]]:
        """Create a model instance for the given model ID."""
        params = {"temperature": 0.7, "streaming": True}
        if input_params:
            params.update(input_params)

        # Get model info from the registry cache
        model_info = await self.langgate_client.get_llm_info(model_id)

        # Transform parameters using the transformer client
        # If switching to using the proxy, you would remove this line
        # and let the proxy handle the parameter transformation instead.
        api_format, model_params = await self.langgate_client.get_params(
            model_id, params
        )
        # api_format defaults to the provider id unless specified in the config.
        # e.g. Specify "openai" for OpenAI-compatible APIs, etc.
        print("API format:", api_format)
        pp(model_params)

        # Get the appropriate model class based on provider
        client_cls_key = ModelProviderId(api_format)
        model_class = MODEL_CLASS_MAP.get(client_cls_key)
        if not model_class:
            raise ValueError(f"No model class for provider {model_info.provider.id}")

        # Create model instance with parameters
        model = model_class(**model_params)

        # Create model info dict
        model_metadata = model_info.model_dump(exclude_none=True)

        return model, model_metadata

In [7]:
model_factory = ModelFactory()
model_id = "openai/gpt-5"
model, model_info = await model_factory.create_model(model_id, {"temperature": 0.7})
model, model_info

API format: openai
{'api_key': SecretStr('**********'),
 'base_url': 'https://api.openai.com/v1',
 'include_reasoning': True,
 'model': 'gpt-5',
 'streaming': True}


/Users/saran/langgate/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3547: UserWarning: WARNING! include_reasoning is not default parameter.
                include_reasoning was transferred to model_kwargs.
                Please confirm that include_reasoning is what you intended.
  await eval(code_obj, self.user_global_ns, self.user_ns)


(ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10cbacec0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10cbad940>, root_client=<openai.OpenAI object at 0x10c40e270>, root_async_client=<openai.AsyncOpenAI object at 0x10cbad6a0>, model_name='gpt-5', model_kwargs={'include_reasoning': True}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', streaming=True),
 {'id': 'openai/gpt-5',
  'name': 'GPT-5',
  'provider_id': 'openai',
  'description': 'GPT-5 is OpenAI\'s most advanced model, offering major improvements in reasoning, code quality, and user experience. It is optimized for complex tasks that require step-by-step reasoning, instruction following, and accuracy in high-stakes use cases. It supports user-specified intent like "think hard about this." Improvements include reductions in hallucination, sycophancy, and better performance in coding, writing, and health-re

### Model Factory Pattern (Decoupled)
- The `LangGateLocal` client is composed of the `LocalRegistryClient` and `LocalTransformerClient`.
- This example is the same as the above, but with each adapter passed in separately to the factory.
 


For example, you may want to sync model data from the registry to a database, along with additional fields.
You could create a `PgModelRegistry` that implements `RegistryClientProtocol`. This would ensure consistent typing throughout your codebase, and you could apply any app-specific validation rules or transformations to the model data before it is sent to the database. For example, limiting the synced models to a subset of the registry models, or transforming the model data to match the database schema. This would allow you to use the same `LangGateLocal` client, while still being able to customize the behavior of each adapter.

In [8]:
from typing import Any
from pprint import pprint as pp

from langchain.chat_models.base import BaseChatModel
from langchain.chat_models import init_chat_model

from langgate.registry import RegistryClientProtocol, LocalRegistryClient
from langgate.transform import LocalTransformerClient, TransformerClientProtocol


class ModelFactoryDecoupled:
    """
    Factory for creating a Langchain `BaseChatModel` instance
    with paramaters from LangGate. Adapters for the registry and transformer
    clients are injected separately, allowing for more flexibility.
    """

    DEFAULT_APP_PARAMS: dict[str, Any] = {
        "streaming": True,
        "verbose": True,
        "temperature": 0.7,
    }

    def __init__(
        self,
        model_registry: RegistryClientProtocol | None = None,
        param_transformer: TransformerClientProtocol | None = None,
    ):
        self.model_registry = model_registry or LocalRegistryClient()
        self.param_transformer = param_transformer or LocalTransformerClient()

    async def create_model(
        self, model_id: str, input_params: dict[str, Any] | None = None
    ) -> tuple[BaseChatModel, dict[str, Any]]:
        """Create a model instance for the given model ID."""
        params = self.DEFAULT_APP_PARAMS.copy()
        if input_params:
            params.update(input_params)

        # Get model info from the registry cache
        model_info = await self.model_registry.get_llm_info(model_id)

        # Transform parameters using the transformer client
        # If switching to using the proxy, you would remove this line
        # and let the proxy handle the parameter transformation instead.
        _, model_params = await self.param_transformer.get_params(model_id, params)
        pp(model_params)

        # Create the appropriate model instance based on provider
        model = init_chat_model(
            model=model_params["model"],
            model_provider=model_info.provider.id,
            kwargs=model_params,
        )
        # Note the use of `langchain.chat_models.init_chat_model` to create the model instance here
        # is more concise and idiomatic but less flexible than the previous explicit mapping of providers to classes.
        # For example, many inference services use the OpenAI API format. `init_chat_model` would
        # not automatically map unknown providers to `ChatOpenAI`. It would raise an error.
        # If you want to use `init_chat_model` with unknown providers, we recommend adding a mapping
        # for the model in `langgate_config.yaml, such as `langchain_class: "ChatOpenAI"`.
        # TODO: We may add this as a built-in feature in future.
        if not model:
            raise ValueError(f"No model class for provider {model_info.provider.id}")

        # Create model info dict
        model_metadata = model_info.model_dump(exclude_none=True)

        return model, model_metadata

In [9]:
model_factory = ModelFactoryDecoupled()
model_id = "openai/gpt-5"
model, model_info = await model_factory.create_model(model_id, {"temperature": 0.7})
model

{'api_key': SecretStr('**********'),
 'base_url': 'https://api.openai.com/v1',
 'include_reasoning': True,
 'model': 'gpt-5',
 'streaming': True,
 'verbose': True}


/var/folders/m_/d_sz9h1577l6qc91t18wm3yh0000gn/T/ipykernel_17963/2849443794.py:50: UserWarning: WARNING! kwargs is not default parameter.
                kwargs was transferred to model_kwargs.
                Please confirm that kwargs is what you intended.
  model = init_chat_model(


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10cd50050>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10cd50a50>, root_client=<openai.OpenAI object at 0x10c7b3d90>, root_async_client=<openai.AsyncOpenAI object at 0x10cd507d0>, model_name='gpt-5', model_kwargs={'kwargs': {'streaming': True, 'verbose': True, 'api_key': SecretStr('**********'), 'base_url': 'https://api.openai.com/v1', 'include_reasoning': True, 'model': 'gpt-5'}}, openai_api_key=SecretStr('**********'))